### Problem 1

a) Given the regression models for the response $Y$ and single covariate $x$, we can write the following as linear regression models as is is (i):

$$\text{2.} \quad Y = \beta_0 + \beta_1 \frac{1}{x} + \beta_2 x^2 + \varepsilon$$

If a parameter is kept fixed (ii):

$$\text{4.} \quad Y = \beta_0 + \beta_1 x^{\beta_2} + \varepsilon \ | \ \beta_2 \text{ fixed}$$

After a suitable transformation (iii):
\begin{align*}\text{3.} \quad Y &= \frac{1}{\beta_0 + \beta_1 x + \beta_2 x^2 + \varepsilon} \\
&\Downarrow \ P = (Y^{-1}) \\
P &= \beta_0 + \beta_1 x + \beta_2 x^2 + \varepsilon \end{align*}

\begin{align*}\text{5.} \quad Y &= \beta_0 x^{\beta_1} \varepsilon\\
&\Downarrow \ P = (\ln{Y})\\
P &= \ln(\beta_0) + \beta_1 \ln(x) + \hat{\varepsilon} \end{align*}

Note: For 5. the model is linear after the $\ln$-transformation, however $\hat{\varepsilon}$ might be from another distribution than the original $\varepsilon$.

b) For the models that can be used, their dimensions will be
* For 2. $\quad \mathbf{X}: \ (N \times 3)$ matrix (with intercept), $\quad \beta: \ (3 \times 1)$ vector
* For 4. $\quad \mathbf{X}: \ (N \times 2)$ matrix (with intercept), $\quad \beta: \ (2 \times 1)$ vector
* For 3. $\quad \mathbf{X}: \ (N \times 3)$ matrix (with intercept), $\quad \beta: \ (3 \times 1)$ vector
* For 5. $\quad \mathbf{X}: \ (N \times 2)$ matrix (with intercept), $\quad \beta: \ (2 \times 1)$ vector

### Problem 2

a)

In [65]:
nuclear.data <- read.table("https://www.uio.no/studier/emner/matnat/math/STK2100/data/nuclear.dat", sep = "\t", header = FALSE)

In [66]:
attach(nuclear.data)
head(nuclear.data)

The following objects are masked from nuclear.data (pos = 3):

    V1, V10, V11, V2, V3, V4, V5, V6, V7, V8, V9




,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,cost,date,t1,t2,cap,pr,ne,ct,bw,cum.n,pt
2,460.05,68.58,14,46,687,0,1,0,0,14,0
3,452.99,67.33,10,73,1065,0,0,1,0,1,0
4,443.22,67.33,10,85,1065,1,0,1,0,1,0
5,652.32,68,11,67,1065,0,1,1,0,12,0
6,642.23,68,11,78,1065,1,1,1,0,12,0


In [67]:
# Converting first row to column names, and then deleting it
colnames(nuclear.data) <- nuclear.data[1, ]
nuclear.data <- nuclear.data[-1, ]
head(nuclear.data)

,cost,date,t1,t2,cap,pr,ne,ct,bw,cum.n,pt
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,460.05,68.58,14,46,687,0,1,0,0,14,0
3,452.99,67.33,10,73,1065,0,0,1,0,1,0
4,443.22,67.33,10,85,1065,1,0,1,0,1,0
5,652.32,68,11,67,1065,0,1,1,0,12,0
6,642.23,68,11,78,1065,1,1,1,0,12,0
7,345.39,67.92,13,51,514,0,1,1,0,3,0


```

In [68]:
nuclear.data <- data.frame(apply(nuclear.data, 2, as.numeric))
nuclear.fit <- lm(log(cost) ~ . , data = nuclear.data)


In [69]:
summary(nuclear.fit)


Call:
lm(formula = log(cost) ~ ., data = nuclear.data)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.284032 -0.081677  0.009502  0.090890  0.266548 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.063e+01  5.710e+00  -1.862  0.07662 .  
date         2.276e-01  8.656e-02   2.629  0.01567 *  
t1           5.252e-03  2.230e-02   0.236  0.81610    
t2           5.606e-03  4.595e-03   1.220  0.23599    
cap          8.837e-04  1.811e-04   4.878 7.99e-05 ***
pr          -1.081e-01  8.351e-02  -1.295  0.20943    
ne           2.595e-01  7.925e-02   3.274  0.00362 ** 
ct           1.155e-01  7.027e-02   1.644  0.11503    
bw           3.680e-02  1.063e-01   0.346  0.73261    
cum.n       -1.203e-02  7.828e-03  -1.536  0.13944    
pt          -2.220e-01  1.304e-01  -1.702  0.10352    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1697 on 21 degrees of freedom
Multiple R-squared:  0.8635,	A

In [70]:
# constructing 95% CI for betas corresponding to t1, t2, bw

N <- nrow(nuclear.data) # data (rows)
p <- length(coef(nuclear.fit)) # covariates (columns)
beta_indices <- c(3, 4, 9) #t1, t2, bw
X <- model.matrix(nuclear.fit) # design matrix X from nuclear.fit
v <- solve(t(X) %*% X) 

sigma.hat <- sqrt(sum(nuclear.fit$residuals^2) / (N - p - 1)) # Standardavviket til residualene
beta.hat <- coef(nuclear.fit)[beta_indices]
se.beta.hat <- sigma.hat * sqrt(diag(v)[beta_indices])

# CI for the chosen betas
#beta.hat + se.beta.hat * qt(c(0.025, 0.975), N - p - 1)
confint(nuclear.fit, level=0.95)[beta_indices, ]

,2.5 %,97.5 %
t1,-0.041123331,0.05162679
t2,-0.003949911,0.01516185
bw,-0.184211843,0.25780411


That the variables `t1, t2, bw` does not have significant importance, is now both seen in the table summary, as well as with 95% confidence here.

b) We have that a $(1 - \alpha) \cdot 100\%$ prediction interval for $Y$ is given by

$$\hat{y} \pm t_{1-\alpha/2; N-p-1} \hat{\sigma} \sqrt {1 + \mathbf{x}^T(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{x}}$$

Since, in our case, $Y = \log(Z)$ we have that $Z = \exp(Y)$


$$\exp \left( \hat{y} \pm t_{1-\alpha/2; N-p-1} \hat{\sigma} \sqrt {1 + \mathbf{x}^T(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{x}}\right)$$